# PyEPICS Regression Tests & Validation Notebook

This notebook performs the following:
1. Discovers and runs existing **unit tests**
2. Parses EEDL / EPDL / EADL files and produces **cross-section plots**
3. Compares results against **NIST / EPICS 2023 reference data**
4. Verifies that **data dictionaries** from PyEEDL are present in PyEPICS

> **Tip:** Run `python tests/generate_report.py` to produce a self-contained PDF report
> with all analyses, plots, and pass/fail summaries.

---

## 1. Setup & Imports

In [ ]:
import sys, os, glob, ast, inspect, importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add PyEPICS to path
PYEPICS_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PYEPICS_ROOT not in sys.path:
    sys.path.insert(0, PYEPICS_ROOT)

# Add PyEEDL to path (for comparison)
PYEEDL_ROOT = os.path.abspath(os.path.join(PYEPICS_ROOT, "..", "PyEEDL"))

from pyepics.readers.eedl import EEDLReader
from pyepics.readers.epdl import EPDLReader
from pyepics.readers.eadl import EADLReader
from pyepics.converters.hdf5 import convert_dataset_to_hdf5
from pyepics.utils.constants import (
    PERIODIC_TABLE, MF_MT, PHOTON_MF_MT, ATOMIC_MF_MT,
    MF23, MF26, MF27, MF28,
    SECTIONS_ABBREVS, PHOTON_SECTIONS_ABBREVS, ATOMIC_SECTIONS_ABBREVS,
    SUBSHELL_LABELS, SUBSHELL_DESIGNATORS,
    FINE_STRUCTURE, ELECTRON_MASS, BARN_TO_CM2,
)

plt.rcParams.update({
    "figure.figsize": (12, 7),
    "font.size": 12,
    "axes.grid": True,
    "grid.alpha": 0.3,
})

print(f"PyEPICS root: {PYEPICS_ROOT}")
print(f"PyEEDL root:  {PYEEDL_ROOT}")
print("Setup OK")

PyEPICS root: /Users/melekderman/github/Summer25/3_MCDC-Electron/DrPrinja/PyEPICS
PyEEDL root:  /Users/melekderman/github/Summer25/3_MCDC-Electron/DrPrinja/PyEEDL
Setup OK ✓


## 2. Discover & List Existing Unit Tests

Scans both repositories for `test_*.py` files and lists test classes and functions.

In [ ]:
def discover_tests(root_dir, label):
    """Scan test files via AST and extract class/function names."""
    rows = []
    test_files = sorted(glob.glob(os.path.join(root_dir, "**", "test_*.py"), recursive=True))
    for fpath in test_files:
        rel = os.path.relpath(fpath, root_dir)
        try:
            tree = ast.parse(open(fpath).read())
        except SyntaxError:
            rows.append({"repo": label, "file": rel, "class": "PARSE ERROR", "function": ""})
            continue
        for node in ast.walk(tree):
            if isinstance(node, ast.ClassDef) and node.name.startswith("Test"):
                for item in node.body:
                    if isinstance(item, (ast.FunctionDef, ast.AsyncFunctionDef)) and item.name.startswith("test_"):
                        rows.append({"repo": label, "file": rel, "class": node.name, "function": item.name})
            elif isinstance(node, ast.FunctionDef) and node.name.startswith("test_") and not any(
                isinstance(p, ast.ClassDef) for p in ast.walk(tree) if hasattr(p, 'body') and node in getattr(p, 'body', [])
            ):
                rows.append({"repo": label, "file": rel, "class": "(module-level)", "function": node.name})
    return rows

rows_epics = discover_tests(PYEPICS_ROOT, "PyEPICS")
rows_eedl = discover_tests(PYEEDL_ROOT, "PyEEDL")

df_tests = pd.DataFrame(rows_epics + rows_eedl)
print(f"PyEPICS test functions: {len(rows_epics)}")
print(f"PyEEDL  test functions: {len(rows_eedl)}")
print()

# Display PyEPICS tests as a table
if rows_epics:
    display(pd.DataFrame(rows_epics).drop(columns="repo"))
else:
    print("No tests found in PyEPICS.")

PyEPICS test fonksiyonu: 74
PyEEDL  test fonksiyonu: 0



,file,class,function
0,tests/test_eadl.py,TestEADLDataset,test_atomic_number
1,tests/test_eadl.py,TestEADLDataset,test_symbol
2,tests/test_eadl.py,TestEADLDataset,test_subshell_count
3,tests/test_eadl.py,TestEADLDataset,test_k_shell_exists
4,tests/test_eadl.py,TestEADLDataset,test_k_binding_energy
...,...,...,...
69,tests/test_hdf5.py,TestWriteEADL,test_binding_energy_value
70,tests/test_hdf5.py,TestWriteEADL,test_radiative_transitions
71,tests/test_hdf5.py,TestWriteEADL,test_summary_arrays
72,tests/test_hdf5.py,TestConvertErrors,test_invalid_dataset_type


## 3. Run Unit Tests

Runs tests programmatically via `pytest` and displays the results.

In [ ]:
import pytest

# Run pytest programmatically
test_dir = os.path.join(PYEPICS_ROOT, "tests")
exit_code = pytest.main([test_dir, "-v", "--tb=short", "-q"])

status_map = {0: "ALL PASSED", 1: "FAILURES", 2: "ERROR", 5: "NO TESTS"}
print(f"\n{'='*60}")
print(f"pytest exit code: {exit_code} -> {status_map.get(exit_code, 'UNKNOWN')}")
print(f"{'='*60}")

============================= test session starts ==============================
platform darwin -- Python 3.13.5, pytest-9.0.2, pluggy-1.6.0
rootdir: /Users/melekderman/github/Summer25/3_MCDC-Electron/DrPrinja/PyEPICS/tests
collected 74 items

test_eadl.py ...........                                                 [ 14%]
test_eedl.py ......................................                      [ 66%]
test_epdl.py ..........                                                  [ 79%]
test_hdf5.py ...............                                             [100%]

============================== 74 passed in 0.13s ==============================

pytest exit code: 0 → ALL PASSED ✓


## 4. Regression Tests: EEDL Electron Cross-Section Plots

Parses ENDF files with `EEDLReader` and plots electron interaction cross sections.
Each element shows total, elastic, bremsstrahlung, excitation, and ionization on a log-log scale.

In [ ]:
# Locate EEDL files
eedl_dir = os.path.join(PYEEDL_ROOT, "eedl")
eedl_files = sorted(glob.glob(os.path.join(eedl_dir, "*.endf")))
print(f"EEDL files found: {len(eedl_files)}")

if not eedl_files:
    print("WARNING: No EEDL ENDF files found.")
    print("  Please download via PyEEDL/download_data.py or from IAEA:")
    print("  https://www-nds.iaea.org/epics/")
else:
    for f in eedl_files[:5]:
        print(f"  {os.path.basename(f)}")

EEDL dosyaları bulundu: 0
⚠ EEDL ENDF dosyaları bulunamadı.
  Lütfen PyEEDL/download_data.py veya IAEA'dan indirin:
  https://www-nds.iaea.org/epics/


In [ ]:
def plot_eedl_cross_sections(endf_path):
    """Parse an EEDL ENDF file and plot all cross sections."""
    reader = EEDLReader()
    ds = reader.read(endf_path)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Style map: abbreviation -> plot style
    styles = {
        "xs_tot":  {"color": "black",  "ls": "--", "lw": 2,   "label": "Total"},
        "xs_el":   {"color": "blue",   "ls": "-",  "lw": 1.5, "label": "Elastic"},
        "xs_lge":  {"color": "purple", "ls": ":",  "lw": 1,   "label": "Large Angle"},
        "xs_brem": {"color": "red",    "ls": "-",  "lw": 1.5, "label": "Bremsstrahlung"},
        "xs_exc":  {"color": "green",  "ls": "-",  "lw": 1.5, "label": "Excitation"},
        "xs_ion":  {"color": "orange", "ls": "-",  "lw": 1.5, "label": "Ionization"},
    }
    
    for abbrev, xs in ds.cross_sections.items():
        sty = dict(styles.get(abbrev, {"color": "gray", "ls": "-", "lw": 0.8, "label": abbrev}))
        if len(xs.energy) > 0 and len(xs.cross_section) > 0:
            ax.loglog(xs.energy, xs.cross_section, label=sty.pop("label", abbrev), **sty)
    
    ax.set_xlabel("Energy (eV)")
    ax.set_ylabel("Cross Section (barns)")
    ax.set_title(f"EEDL Electron Cross Sections — {ds.symbol} (Z={ds.Z})")
    ax.legend(loc="upper right")
    ax.grid(True, which="both", alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return ds

# H (Z=1)
h_files = [f for f in eedl_files if "001000" in f or "ZA001" in f]
if h_files:
    ds_h = plot_eedl_cross_sections(h_files[0])
else:
    print("H (Z=1) EEDL file not found")

In [ ]:
# Fe (Z=26)
fe_files = [f for f in eedl_files if "026000" in f or "ZA026" in f]
if fe_files:
    ds_fe = plot_eedl_cross_sections(fe_files[0])
else:
    print("Fe (Z=26) EEDL file not found")

### 4b. EPDL Photon Cross-Section Plots — Fe (Z=26)

In [ ]:
# Locate EPDL files
epdl_dir = os.path.join(PYEEDL_ROOT, "eedl")
epdl_files = sorted(glob.glob(os.path.join(epdl_dir, "*EPDL*")))
if not epdl_files:
    epdl_files = sorted(glob.glob(os.path.join(PYEEDL_ROOT, "**", "*EPDL*"), recursive=True))

print(f"EPDL files found: {len(epdl_files)}")

def plot_epdl_cross_sections(endf_path):
    """Parse an EPDL ENDF file and plot photon cross sections."""
    reader = EPDLReader()
    ds = reader.read(endf_path)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    
    colors = ["black", "blue", "red", "green", "orange", "purple", "cyan", "magenta"]
    for i, (abbrev, xs) in enumerate(ds.cross_sections.items()):
        c = colors[i % len(colors)]
        ls = "--" if "tot" in abbrev else "-"
        if len(xs.energy) > 0:
            ax.loglog(xs.energy, xs.cross_section, label=abbrev, color=c, ls=ls)
    
    ax.set_xlabel("Energy (eV)")
    ax.set_ylabel("Cross Section (barns)")
    ax.set_title(f"EPDL Photon Cross Sections — {ds.symbol} (Z={ds.Z})")
    ax.legend(loc="upper right")
    ax.grid(True, which="both", alpha=0.3)
    plt.tight_layout()
    plt.show()
    return ds

fe_epdl = [f for f in epdl_files if "026000" in f or "ZA026" in f]
if fe_epdl:
    ds_fe_ph = plot_epdl_cross_sections(fe_epdl[0])
else:
    print("Fe EPDL file not found")

## 5. Comparison Against EPICS Reference Data

### 5a. K-Shell Binding Energy: EADL vs NIST Reference

Compares K-shell binding energies parsed from EADL against NIST reference values.
Produces both an overlay plot and a relative-error bar chart.

In [ ]:
# --- Extract binding energies from EADL / MC/DC HDF5 files ---
eadl_files = sorted(glob.glob(os.path.join(PYEEDL_ROOT, "eedl", "*EADL*")))
if not eadl_files:
    eadl_files = sorted(glob.glob(os.path.join(PYEEDL_ROOT, "**", "*EADL*"), recursive=True))

print(f"EADL files found: {len(eadl_files)}")

# Alternative: read binding energies from MC/DC HDF5 files
mcdc_dir = os.path.join(PYEEDL_ROOT, "mcdc_data")
h5_files = sorted(glob.glob(os.path.join(mcdc_dir, "*.h5")))
print(f"MC/DC HDF5 files: {len(h5_files)}")

import h5py

be_rows = []
for h5path in h5_files:
    try:
        with h5py.File(h5path, "r") as f:
            Z = int(f["atomic_number"][()])
            sym = PERIODIC_TABLE.get(Z, {}).get("symbol", "?")
            if "electron_reactions/ionization/subshells" in f:
                for shell_name, grp in f["electron_reactions/ionization/subshells"].items():
                    be = float(grp["binding_energy"][()])
                    be_rows.append({"Z": Z, "symbol": sym, "subshell": shell_name, "be_eV": be})
    except Exception as e:
        print(f"  Error reading {os.path.basename(h5path)}: {e}")

df_be = pd.DataFrame(be_rows)
print(f"\nTotal subshell records: {len(df_be)}")
df_be.head(10)

In [ ]:
# --- Load NIST reference data ---
ref_csv = os.path.join(PYEPICS_ROOT, "reference_data", "reference_binding_energies.csv")
df_ref = pd.read_csv(ref_csv)
df_ref_k = df_ref[df_ref["subshell"] == "K"].copy()

# EADL K-shell data
df_k = df_be[df_be["subshell"] == "K"].sort_values("Z").copy()

# --- Comparison plot ---
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True,
                                gridspec_kw={"height_ratios": [3, 1]})

ax1.semilogy(df_k["Z"], df_k["be_eV"], "o-", color="blue", markersize=4,
             label="EADL (parsed)", alpha=0.8)
ax1.semilogy(df_ref_k["Z"], df_ref_k["binding_energy_eV"], "s", color="red",
             markersize=8, label="NIST Reference", zorder=5)
ax1.set_ylabel("K-Shell Binding Energy (eV)")
ax1.set_title("K-Shell Binding Energy: EADL vs NIST Reference")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Compute relative error (merge on Z)
df_comp = pd.merge(df_k[["Z", "be_eV"]], df_ref_k[["Z", "binding_energy_eV"]], on="Z")
df_comp["rel_error_pct"] = 100 * abs(df_comp["be_eV"] - df_comp["binding_energy_eV"]) / df_comp["binding_energy_eV"]

ax2.bar(df_comp["Z"], df_comp["rel_error_pct"], color="orange", alpha=0.7, width=1.0)
ax2.axhline(5.0, color="red", ls="--", label="5% threshold")
ax2.set_ylabel("Relative Error (%)")
ax2.set_xlabel("Atomic Number (Z)")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

max_err = df_comp["rel_error_pct"].max()
print(f"Max relative error: {max_err:.2f}%")
if max_err < 5.0:
    print("PASS: All K-shell binding energies within 5% of NIST values")
else:
    print(f"FAIL: Some values exceed 5% threshold!")

### 5b. K -> L2 Transition Energies

Computes transition energies from K and L2 binding-energy differences.

In [ ]:
df_k_z = df_be[df_be["subshell"] == "K"].set_index("Z")["be_eV"]
df_l2_z = df_be[df_be["subshell"] == "L2"].set_index("Z")["be_eV"]
df_l3_z = df_be[df_be["subshell"] == "L3"].set_index("Z")["be_eV"]

# Common Z values
common_kl2 = df_k_z.index.intersection(df_l2_z.index)
common_kl3 = df_k_z.index.intersection(df_l3_z.index)

trans_kl2 = (df_k_z.loc[common_kl2] - df_l2_z.loc[common_kl2]).reset_index()
trans_kl2.columns = ["Z", "transition_eV"]

trans_kl3 = (df_k_z.loc[common_kl3] - df_l3_z.loc[common_kl3]).reset_index()
trans_kl3.columns = ["Z", "transition_eV"]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 9), sharex=True)

ax1.loglog(trans_kl2["Z"], trans_kl2["transition_eV"], "o-", color="blue",
           markersize=4, label="K -> L2")
ax1.loglog(trans_kl3["Z"], trans_kl3["transition_eV"], "s-", color="red",
           markersize=4, label="K -> L3", alpha=0.7)
ax1.set_ylabel("Transition Energy (eV)")
ax1.set_title("Atomic Subshell Transition Energies from EADL")
ax1.legend()
ax1.grid(True, which="both", alpha=0.3)
ax1.set_xlim(4, 100)

# L2-L3 splitting
splitting = (df_l2_z.loc[common_kl3] - df_l3_z.loc[common_kl3]).reset_index()
splitting.columns = ["Z", "splitting_eV"]
ax2.semilogy(splitting["Z"], splitting["splitting_eV"], "^-", color="green",
             markersize=4, label="L2 - L3 splitting")
ax2.set_xlabel("Atomic Number (Z)")
ax2.set_ylabel("L2-L3 Splitting (eV)")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"K -> L2 transition energies: {len(trans_kl2)} elements")
print(f"K -> L3 transition energies: {len(trans_kl3)} elements")

### 5c. HDF5 Cross-Section Plots (from MC/DC HDF5 files)

Reads cross sections from PyEEDL's MC/DC HDF5 output files and produces plots in the same style as PyEEDL's `test_be.ipynb`.

In [ ]:
def plot_h5_cross_sections(h5_path, element_label=None):
    """Plot all electron cross sections from an MC/DC HDF5 file."""
    with h5py.File(h5_path, "r") as f:
        Z = int(f["atomic_number"][()])
        sym = element_label or PERIODIC_TABLE.get(Z, {}).get("symbol", f"Z={Z}")
        e_grid = f["electron_reactions/xs_energy_grid"][()]
        
        reactions = {}
        for name, path in [
            ("Total",          "electron_reactions/total/xs"),
            ("Elastic",        "electron_reactions/elastic_scattering/xs"),
            ("Large Angle",    "electron_reactions/elastic_scattering/large_angle/xs"),
            ("Small Angle",    "electron_reactions/elastic_scattering/small_angle/xs"),
            ("Bremsstrahlung", "electron_reactions/bremsstrahlung/xs"),
            ("Excitation",     "electron_reactions/excitation/xs"),
            ("Ionization",     "electron_reactions/ionization/xs"),
        ]:
            if path in f:
                reactions[name] = f[path][()]
    
    fig, ax = plt.subplots(figsize=(14, 8))
    
    styles = {
        "Total":          {"color": "black",  "ls": "--", "lw": 2},
        "Elastic":        {"color": "blue",   "ls": "-",  "lw": 1.5},
        "Large Angle":    {"color": "purple", "ls": ":",  "lw": 1.2},
        "Small Angle":    {"color": "cyan",   "ls": ":",  "lw": 1.2},
        "Bremsstrahlung": {"color": "red",    "ls": "-",  "lw": 1.5},
        "Excitation":     {"color": "green",  "ls": "-",  "lw": 1.5},
        "Ionization":     {"color": "orange", "ls": "-",  "lw": 1.5},
    }
    
    for name, xs in reactions.items():
        sty = styles.get(name, {"color": "gray", "ls": "-", "lw": 1})
        ax.loglog(e_grid, xs, label=name, **sty)
    
    ax.set_xlabel("Energy (eV)")
    ax.set_ylabel("Cross Section (barns)")
    ax.set_title(f"MC/DC Electron Cross Sections — {sym} (Z={Z})")
    ax.legend(loc="upper right")
    ax.grid(True, which="both", alpha=0.3)
    plt.tight_layout()
    plt.show()

# Plot H, Fe, Al
for elem in ["H", "Fe", "Al"]:
    h5 = os.path.join(mcdc_dir, f"{elem}.h5")
    if os.path.exists(h5):
        plot_h5_cross_sections(h5, elem)
    else:
        print(f"{elem}.h5 not found")

## 6. PyEEDL Data Structure Inspection: Descriptions & Metadata

Inspects the data dictionaries (MF23, MF26, MF27, MF28, MF_MT, etc.) from PyEEDL's `data.py` and compares them with PyEPICS's `constants.py`.

In [ ]:
# Exported symbols from PyEEDL data.py
pyeedl_data_path = os.path.join(PYEEDL_ROOT, "pyeedl", "data.py")
pyeedl_exports = []
if os.path.exists(pyeedl_data_path):
    tree = ast.parse(open(pyeedl_data_path).read())
    for node in ast.walk(tree):
        if isinstance(node, ast.Assign):
            for target in node.targets:
                if isinstance(target, ast.Name) and target.id.isupper():
                    pyeedl_exports.append(target.id)
    print("PyEEDL data.py exports:")
    for name in sorted(set(pyeedl_exports)):
        print(f"  - {name}")
else:
    print("PyEEDL data.py not found")

print()

# Exported symbols from PyEPICS constants.py
from pyepics.utils import constants as epics_const
pyepics_exports = [n for n in dir(epics_const) if n.isupper() and not n.startswith("_")]
print("PyEPICS constants.py exports:")
for name in sorted(pyepics_exports):
    print(f"  - {name}")

## 7. PyEPICS Data Descriptions & Documentation Check

Compares data dictionaries (MF_MT, PERIODIC_TABLE, MF23, MF26, etc.) side by side and detects missing keys.

In [ ]:
# --- Value comparison: MF_MT dictionaries ---
# Import PyEEDL's data.py
pyeedl_data_spec = importlib.util.spec_from_file_location("pyeedl_data", pyeedl_data_path)
pyeedl_data = importlib.util.module_from_spec(pyeedl_data_spec)
pyeedl_data_spec.loader.exec_module(pyeedl_data)

comparison_rows = []

dict_pairs = [
    ("MF_MT",                 getattr(pyeedl_data, "MF_MT", {}),                 MF_MT),
    ("SECTIONS_ABBREVS",      getattr(pyeedl_data, "SECTIONS_ABBREVS", {}),      SECTIONS_ABBREVS),
    ("PHOTON_MF_MT",          getattr(pyeedl_data, "PHOTON_MF_MT", {}),          PHOTON_MF_MT),
    ("PHOTON_SECTIONS_ABBREVS", getattr(pyeedl_data, "PHOTON_SECTIONS_ABBREVS", {}), PHOTON_SECTIONS_ABBREVS),
    ("ATOMIC_MF_MT",          getattr(pyeedl_data, "ATOMIC_MF_MT", {}),          ATOMIC_MF_MT),
    ("SUBSHELL_LABELS",       getattr(pyeedl_data, "SUBSHELL_LABELS", {}),       SUBSHELL_LABELS),
    ("SUBSHELL_DESIGNATORS",  getattr(pyeedl_data, "SUBSHELL_DESIGNATORS", {}),  SUBSHELL_DESIGNATORS),
    ("PERIODIC_TABLE",        getattr(pyeedl_data, "PERIODIC_TABLE", {}),        PERIODIC_TABLE),
    ("MF23",                  getattr(pyeedl_data, "MF23", {}),                  MF23),
    ("MF26",                  getattr(pyeedl_data, "MF26", {}),                  MF26),
    ("MF27",                  getattr(pyeedl_data, "MF27", {}),                  MF27),
    ("MF28",                  getattr(pyeedl_data, "MF28", {}),                  MF28),
]

for name, old_dict, new_dict in dict_pairs:
    old_keys = set(old_dict.keys())
    new_keys = set(new_dict.keys())
    missing = old_keys - new_keys
    extra = new_keys - old_keys
    matching = old_keys & new_keys
    
    status = "Full match" if not missing else f"Missing: {len(missing)} keys"
    comparison_rows.append({
        "Dictionary": name,
        "PyEEDL keys": len(old_keys),
        "PyEPICS keys": len(new_keys),
        "Matching": len(matching),
        "Missing": len(missing),
        "Extra": len(extra),
        "Status": status,
    })

df_comp_dicts = pd.DataFrame(comparison_rows)
display(df_comp_dicts)

## 8. Missing Documentation & Metadata Detection

Checks docstring coverage across PyEPICS modules. Reports functions/classes that lack docstrings.

In [ ]:
def check_docstrings(root_dir, package_name):
    """Check all .py files for function/class docstrings."""
    rows = []
    py_files = sorted(glob.glob(os.path.join(root_dir, package_name, "**", "*.py"), recursive=True))
    
    for fpath in py_files:
        rel = os.path.relpath(fpath, root_dir)
        try:
            tree = ast.parse(open(fpath).read())
        except SyntaxError:
            continue
        
        # Module docstring
        mod_doc = ast.get_docstring(tree)
        rows.append({
            "file": rel, "type": "module", "name": rel,
            "has_docstring": mod_doc is not None,
        })
        
        for node in ast.walk(tree):
            if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef)):
                doc = ast.get_docstring(node)
                rows.append({
                    "file": rel, "type": "function", "name": node.name,
                    "has_docstring": doc is not None,
                })
            elif isinstance(node, ast.ClassDef):
                doc = ast.get_docstring(node)
                rows.append({
                    "file": rel, "type": "class", "name": node.name,
                    "has_docstring": doc is not None,
                })
    return pd.DataFrame(rows)

df_docs = check_docstrings(PYEPICS_ROOT, "pyepics")

# Summary
total = len(df_docs)
with_doc = df_docs["has_docstring"].sum()
without_doc = total - with_doc

print(f"{'='*60}")
print(f"PyEPICS Docstring Coverage Report")
print(f"{'='*60}")
print(f"Total items:          {total}")
print(f"With docstring:       {with_doc} ({100*with_doc/total:.0f}%)")
print(f"Without docstring:    {without_doc} ({100*without_doc/total:.0f}%)")
print()

# Show items without docstrings
missing = df_docs[~df_docs["has_docstring"]]
if len(missing) > 0:
    print("Items missing docstrings:")
    display(missing[["file", "type", "name"]])
else:
    print("All modules, classes, and functions have docstrings!")

## 9. Physical Constants Comparison

Verifies that physical constants in both packages are identical.

In [ ]:
const_names = [
    "FINE_STRUCTURE", "ELECTRON_MASS", "BARN_TO_CM2",
    "PLANCK_CONSTANT", "SPEED_OF_LIGHT", "ELECTRON_CHARGE",
]

const_rows = []
for name in const_names:
    old_val = getattr(pyeedl_data, name, None)
    new_val = getattr(epics_const, name, None)
    match = old_val == new_val if (old_val is not None and new_val is not None) else False
    const_rows.append({
        "Constant": name,
        "PyEEDL": old_val,
        "PyEPICS": new_val,
        "Match": "OK" if match else "MISMATCH",
    })

df_const = pd.DataFrame(const_rows)
display(df_const)

all_match = all(r["Match"] == "OK" for r in const_rows)
print(f"\n{'All physical constants match!' if all_match else 'MISMATCH: some constants differ!'}")

## 10. HDF5 Round-Trip Validation

Writes a dataset with `convert_dataset_to_hdf5()` and reads it back with `h5py` to verify data integrity.

In [ ]:
import tempfile

# Synthetic EEDL dataset for round-trip test
from pyepics.models.records import EEDLDataset, CrossSectionRecord, AverageEnergyLoss
from pyepics.converters.hdf5 import _write_eedl, _write_metadata

energy = np.logspace(1, 7, 200)
xs_total = 1e6 * energy**(-0.8)
xs_elastic = 0.9e6 * energy**(-0.8)
xs_brem = 0.01 * energy**0.2

test_ds = EEDLDataset(
    Z=26,
    symbol="Fe",
    atomic_weight_ratio=55.345,
    ZA=26000.0,
    cross_sections={
        "xs_tot": CrossSectionRecord(label="xs_tot", energy=energy, cross_section=xs_total),
        "xs_el": CrossSectionRecord(label="xs_el", energy=energy, cross_section=xs_elastic),
        "xs_brem": CrossSectionRecord(label="xs_brem", energy=energy, cross_section=xs_brem),
    },
    distributions={},
    average_energy_losses={},
)

# Write to HDF5 using internal writers
with tempfile.NamedTemporaryFile(suffix=".h5", delete=False) as tmp:
    tmp_path = tmp.name

with h5py.File(tmp_path, "w") as h5f:
    _write_metadata(h5f, test_ds)
    _write_eedl(h5f, test_ds)

# Read back and verify
z_grp = f"EEDL/Z_{test_ds.Z:03d}"
checks = []
with h5py.File(tmp_path, "r") as f:
    # Metadata
    checks.append(("Z == 26",          int(f["metadata/Z"][()]) == 26))
    checks.append(("symbol == Fe",     f["metadata/symbol"].asstr()[()] == "Fe"))
    
    # EEDL groups
    checks.append(("EEDL/ group exists",  "EEDL" in f))
    checks.append(("total/ exists",       f"{z_grp}/total" in f))
    checks.append(("elastic_scattering/ exists", f"{z_grp}/elastic_scattering" in f))
    
    # Data match
    rt_energy = f[f"{z_grp}/xs_energy_grid"][:]
    rt_xs = f[f"{z_grp}/total/xs"][:]
    checks.append(("energy shape match",     rt_energy.shape == energy.shape))
    checks.append(("energy allclose",  np.allclose(rt_energy, energy)))
    checks.append(("xs allclose",      np.allclose(rt_xs, xs_total)))
    
    # Unit attribute
    ds_eg = f[f"{z_grp}/xs_energy_grid"]
    checks.append(("energy unit == eV",  ds_eg.attrs.get("units") == "eV"))

os.unlink(tmp_path)

# Display results
print(f"{'Check':<35} {'Result'}")
print("=" * 50)
passed = 0
for name, ok in checks:
    status = "PASS" if ok else "FAIL"
    print(f"{name:<35} {status}")
    if ok:
        passed += 1

print(f"\n{passed}/{len(checks)} checks passed")

## 11. Overall Summary

Aggregated pass/fail status for all validation steps.

In [ ]:
print("=" * 60)
print("  PyEPICS REGRESSION TEST SUMMARY")
print("=" * 60)
print()

summary = {
    "Unit tests (pytest)":           exit_code == 0,
    "HDF5 round-trip":               passed == len(checks),
    "Physical constants match":      all_match,
    "K-shell BE < 5% error":         max_err < 5.0 if 'max_err' in dir() else None,
}

all_ok = True
for label, ok in summary.items():
    if ok is None:
        status = "SKIPPED (no data)"
    elif ok:
        status = "PASS"
    else:
        status = "FAIL"
        all_ok = False
    print(f"  {label:<40} {status}")

print()
if all_ok:
    print("  ALL REGRESSION TESTS PASSED")
else:
    print("  SOME TESTS FAILED")
print("=" * 60)